In [8]:
# Setup

from dolo import *
from dolo.compiler.misc import CalibrationDict, calibration_to_vector
import dolark 
from dolark import HModel
from dolark.equilibrium import find_steady_state
from dolark.perturbation import perturb
from dolo import time_iteration, improved_time_iteration
from matplotlib import pyplot as plt
import numpy as np

In [9]:
#HModel reads the yaml file
aggmodel = HModel('Aiyagari-Copy1.yaml') # default calibration is sigma = 0.2, rho = 0.9, epsilon = 1
aggmodel

In [10]:
eq = find_steady_state(aggmodel)
eq

Computing Initial Initial Rule... 

    calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


done
Computing Steady State...done


In [12]:
df = eq.as_df()
a = df['a']
r = df['r']
w = df['w']
e = df['e']
μ = df['μ']
i = df['i']
    
#setup cnosumption and income
c = np.zeros((len(df),1))
income = np.zeros((len(df),1))
agg_c = 0
agg_inc = 0
# calculate consumption
for j in range(len(df)):
    c[j] = (1+r[j])*a[j] + w[j]*np.exp(e[j]) - i[j]
# calcuate income
for j in range(len(df)):
    income[j] = (r[j]+0.08)*a[j] + w[j]*np.exp(e[j])
# aggregate consumption and aggregat consumption
for j in range(len(df)):
    agg_c = agg_c + c[j]*μ[j]
    agg_inc = agg_inc + income[j]*μ[j]

saving_rate = 1 - agg_c/agg_inc

In [149]:
saving_rate

array([0.2437633])

In [49]:
rows = [['epsilon', 'sig', 'rho','saving rate']]
rho_values = np.linspace(0, 0.9, 4)   #change serial correlation coefficent "rho "in {0, 0.3, 0.6, 0.9}
sig_values = np.linspace(0.2, 0.4, 2) #change the variance of labor shocks "sig" in {0.2, 0.4}
epsilon_values = np.linspace(1, 5, 3)       #change the coefficient of risk aversion {1,3,5}

for l in epsilon_values:
    aggmodel.model.set_calibration( epsilon = l)
    for n in sig_values:
        aggmodel.model.set_calibration( sig = n )
        for m in rho_values:
            aggmodel.model.set_calibration( rho=m )
            eq = find_steady_state(aggmodel)
            df = eq.as_df()
            a = df['a']
            r = df['r']
            w = df['w']
            e = df['e']
            μ = df['μ']
            i = df['i']
    
            #setup cnosumption and income
            c = np.zeros((len(df),1))
            income = np.zeros((len(df),1))
            agg_c = 0
            agg_inc = 0
        
            # calculate consumption
            for j in range(len(df)):
                c[j] = (1+r[j])*a[j] + w[j]*np.exp(e[j]) - i[j]
            # calcuate income
            for j in range(len(df)):
                income[j] = (r[j]+0.08)*a[j] + w[j]*np.exp(e[j])   #0.08 is the depreciation rate
            
            # aggregate consumption and aggregat consumption
            for j in range(len(df)):
                agg_c = agg_c + c[j]*μ[j]
                agg_inc = agg_inc + income[j]*μ[j]
            
            saving_rate= 1 - agg_c/agg_inc
            saving = np.array(saving_rate) 
            
            rows.append((str(l), str(n), str(m), str(saving)))
            
    

Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... done
Computing Steady State...done
Computing Initial Initial Rule... 

In [50]:
rows

[['epsilon', 'sig', 'rho', 'saving rate'],
 ('1.0', '0.2', '0.0', '[0.24275118]'),
 ('1.0', '0.2', '0.3', '[0.24280436]'),
 ('1.0', '0.2', '0.6', '[0.24325385]'),
 ('1.0', '0.2', '0.9', '[0.2437633]'),
 ('1.0', '0.4', '0.0', '[0.24397777]'),
 ('1.0', '0.4', '0.3', '[0.24475714]'),
 ('1.0', '0.4', '0.6', '[0.2465101]'),
 ('1.0', '0.4', '0.9', '[0.24745332]'),
 ('3.0', '0.2', '0.0', '[0.25657752]'),
 ('3.0', '0.2', '0.3', '[0.25732593]'),
 ('3.0', '0.2', '0.6', '[0.26078807]'),
 ('3.0', '0.2', '0.9', '[0.2837639]'),
 ('3.0', '0.4', '0.0', '[0.26799146]'),
 ('3.0', '0.4', '0.3', '[0.27172016]'),
 ('3.0', '0.4', '0.6', '[0.28351045]'),
 ('3.0', '0.4', '0.9', '[0.36529834]'),
 ('5.0', '0.2', '0.0', '[0.26694438]'),
 ('5.0', '0.2', '0.3', '[0.26829488]'),
 ('5.0', '0.2', '0.6', '[0.27408014]'),
 ('5.0', '0.2', '0.9', '[0.31640614]'),
 ('5.0', '0.4', '0.0', '[0.28352314]'),
 ('5.0', '0.4', '0.3', '[0.28936891]'),
 ('5.0', '0.4', '0.6', '[0.30801935]'),
 ('5.0', '0.4', '0.9', '[0.44488939]')]

In [51]:
def pretty_table(rows, column_count, column_spacing=4):
    aligned_columns = []
    for column in range(column_count):
        column_data = list(map(lambda row: row[column], rows))
        aligned_columns.append((max(map(len, column_data)) + column_spacing, column_data))

    for row in range(len(rows)):
        aligned_row = map(lambda x: (x[0], x[1][row]), aligned_columns)
        yield ''.join(map(lambda x: x[1] + ' ' * (x[0] - len(x[1])), aligned_row))

In [52]:
for s in pretty_table(rows, 4):
    print(s)

epsilon    sig    rho    saving rate     
1.0        0.2    0.0    [0.24275118]    
1.0        0.2    0.3    [0.24280436]    
1.0        0.2    0.6    [0.24325385]    
1.0        0.2    0.9    [0.2437633]     
1.0        0.4    0.0    [0.24397777]    
1.0        0.4    0.3    [0.24475714]    
1.0        0.4    0.6    [0.2465101]     
1.0        0.4    0.9    [0.24745332]    
3.0        0.2    0.0    [0.25657752]    
3.0        0.2    0.3    [0.25732593]    
3.0        0.2    0.6    [0.26078807]    
3.0        0.2    0.9    [0.2837639]     
3.0        0.4    0.0    [0.26799146]    
3.0        0.4    0.3    [0.27172016]    
3.0        0.4    0.6    [0.28351045]    
3.0        0.4    0.9    [0.36529834]    
5.0        0.2    0.0    [0.26694438]    
5.0        0.2    0.3    [0.26829488]    
5.0        0.2    0.6    [0.27408014]    
5.0        0.2    0.9    [0.31640614]    
5.0        0.4    0.0    [0.28352314]    
5.0        0.4    0.3    [0.28936891]    
5.0        0.4    0.6    [0.308019

In [19]:
savings

[array([0.24275118]),
 array([0.24280436]),
 array([0.24325385]),
 array([0.2437633]),
 array([0.24397777]),
 array([0.24475714]),
 array([0.2465101]),
 array([0.24745332]),
 array([0.25657752]),
 array([0.25732593]),
 array([0.26078807]),
 array([0.2837639]),
 array([0.26799146]),
 array([0.27172016]),
 array([0.28351045]),
 array([0.36529834]),
 array([0.26694438]),
 array([0.26829488]),
 array([0.27408014]),
 array([0.31640614]),
 array([0.28352314]),
 array([0.28936891]),
 array([0.30801935]),
 array([0.44488939])]

In [21]:
saving = np.array(savings)   #convert array to list

In [23]:
print(saving)

[[0.24275118]
 [0.24280436]
 [0.24325385]
 [0.2437633 ]
 [0.24397777]
 [0.24475714]
 [0.2465101 ]
 [0.24745332]
 [0.25657752]
 [0.25732593]
 [0.26078807]
 [0.2837639 ]
 [0.26799146]
 [0.27172016]
 [0.28351045]
 [0.36529834]
 [0.26694438]
 [0.26829488]
 [0.27408014]
 [0.31640614]
 [0.28352314]
 [0.28936891]
 [0.30801935]
 [0.44488939]]


In [44]:
#rows = [['Term', 'Exponential sum', 'Absolute error', 'Relative error']]
#values = np.linspace(1,5,3)
#for n in values:
   # exp_sum   = n+1
    #abs_err   = abs(n+1)
    #rel_err   = abs(n+2)
    #rows.append((str(n), str(exp_sum), str(abs_err), str(rel_err)))

In [45]:
#rows

[['Term', 'Exponential sum', 'Absolute error', 'Relative error'],
 ('1.0', '2.0', '2.0', '3.0'),
 ('3.0', '4.0', '4.0', '5.0'),
 ('5.0', '6.0', '6.0', '7.0')]

In [46]:
#def pretty_table(rows, column_count, column_spacing=4):
    #aligned_columns = []
    #for column in range(column_count):
        #column_data = list(map(lambda row: row[column], rows))
        #aligned_columns.append((max(map(len, column_data)) + column_spacing, column_data))

    #for row in range(len(rows)):
        #aligned_row = map(lambda x: (x[0], x[1][row]), aligned_columns)
        #yield ''.join(map(lambda x: x[1] + ' ' * (x[0] - len(x[1])), aligned_row))

In [47]:
#for line in pretty_table(rows, 4):
    #print(line)

Term    Exponential sum    Absolute error    Relative error    
1.0     2.0                2.0               3.0               
3.0     4.0                4.0               5.0               
5.0     6.0                6.0               7.0               
